In [5]:
import sys
sys.path.insert(0, '/home/ldoyle/packages')
import h5py
import numpy as np
from tqdm import tqdm
import matplotlib

import matplotlib.dates as mdates
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import pandas as pd
import pysindy as ps
from lr_ed import localreg
from datetime import datetime
from scipy.fft import fft, fftfreq,rfft2

import scipy.signal as signal
from obspy.signal import filter as obsfilt
import glob
from pyproj import Proj


### Load data and Adjust Units

In [41]:
file_list = glob.glob("/home/ldoyle/notebooks/rad_10_smooth_data/*")

In [42]:
n = 1.4682
L = 51.04762

b=0
for q in np.arange(5,8):
    if b==0:
        working_dat = np.load(file_list[q])
        b=1
    else:
        working_dat = np.hstack((working_dat, np.load(file_list[q])))
        
working_dat = ((2*np.pi)/2**16)*((1550.12 * 1e-9)/(0.78 * 4 * np.pi * n * L))*working_dat#[:,::10]
working_dat = working_dat.T *10**6#/(np.std(working_dat))
# working_dat = working_dat[::10,:]
# working_dat = working_dat.reshape(working_dat.shape[0],working_dat.shape[1],1)

In [43]:
print(working_dat.shape)
print(np.amax(working_dat))
print(np.amin(working_dat))

(36000, 5000)
2.0068433398443144
-2.104901850989071


In [5]:
dt=1. #/200
xs = np.arange(working_dat.shape[1]) #2.0419047*np.arange(working_dat.shape[1])
# time = dt*np.arange(working_dat.shape[0])

library_functions = [lambda x: x, lambda x: x * x]
library_function_names = [lambda x: x, lambda x: x + x]
pde_lib = ps.PDELibrary(
    library_functions=library_functions,
    function_names=library_function_names,
    derivative_order=4,
    spatial_grid=xs,
    include_bias=True,
    is_uniform=True,
#     periodic=True
)

In [ ]:
thresh = 0.001
optimizer = ps.STLSQ(threshold=thresh, alpha=1e-5, max_iter=50)
model = ps.SINDy(feature_library=pde_lib, optimizer=optimizer)
model.fit(working_dat, t=dt,ensemble=True)


library_ensemble_coefs = model.coef_list
mean_library_ensemble = np.mean(library_ensemble_coefs, axis=0)
std_library_ensemble = np.std(library_ensemble_coefs, axis=0)

In [ ]:
print("Ensemble STLSQ with Threshold "+str(thresh))
model.print()

### Include depth

In [44]:
latlondepth = np.genfromtxt('south_DAS_latlondepth.txt')

relevant = latlondepth[6558:11558,3]

In [45]:
h_shaped = np.tile(relevant.reshape(-1,1),working_dat.shape[0]).transpose()
data_2 = np.stack((working_dat,h_shaped),
         axis=-1)
# data_2 = np.transpose(np.asarray([working_dat,h_shaped]),[1,2,0])

In [46]:
data_2.shape

(36000, 5000, 2)

In [47]:
inputs_per_library = np.array([[0,1],[0,1],[0,0]])
tensor_array = [[1, 1, 0]]

In [48]:
dt=1. #/200
xs = np.arange(5000) #2.0419047*np.arange(working_dat.shape[1])
# time = dt*np.arange(working_dat.shape[0])

library_functions = [lambda x: x, lambda x: x * x]
library_function_names = [lambda x: x, lambda x: x + x]
pde_lib = ps.PDELibrary(
    library_functions=library_functions,
    function_names=library_function_names,
    derivative_order=4,
    spatial_grid=xs,
    include_bias=True,
    is_uniform=True,
#     periodic=True
)

In [49]:
poly_library = ps.PolynomialLibrary(include_bias=False)
fourier_library = ps.FourierLibrary()

In [50]:
generalized_library = ps.GeneralizedLibrary(
    [poly_library, fourier_library, pde_lib],
    tensor_array=tensor_array,
    inputs_per_library=inputs_per_library,
)

In [51]:
u_dot = ps.FiniteDifference(axis=1)._differentiate(working_dat, t=1)
data_dot = np.stack((u_dot,h_shaped),axis=-1)

In [52]:
optimizer = ps.STLSQ(threshold=5.,alpha=1e-4, normalize_columns=True)
model = ps.SINDy(feature_library=generalized_library, optimizer=optimizer)
model.fit(data_2, x_dot=data_dot)


SINDy(differentiation_method=FiniteDifference(axis=-2),
      feature_library=<pysindy.feature_library.generalized_library.GeneralizedLibrary object at 0x7fb8f1c6f370>,
      feature_names=['x0', 'x1'],
      optimizer=STLSQ(alpha=0.0001, normalize_columns=True, threshold=5.0))

In [53]:
model.print()

(x0)' = -0.207 x0^2 + -2.499 cos(1 x0) + -0.025 sin(1 x1) + -0.100 cos(1 x1) + 2.480 1 + -0.207 x0x0 + 0.006 x0_1 + 0.005 x0_111 + 0.002 x0x0_1 + 0.001 x0x0x0_1 + 0.001 x0x0_11 + -0.006 x0x0_111 + -0.003 x0x0_1111 + -0.946 x0 sin(1 x0) + 0.013 x0 cos(1 x0) + -0.003 x0 sin(1 x1) + -0.004 x0 cos(1 x1) + -0.001 x1 cos(1 x1) + 0.006 x0^2 sin(1 x0) + 0.104 x0^2 cos(1 x0) + -0.001 x0^2 sin(1 x1)
(x1)' = -0.001 x0 + 1.000 x1 + 0.001 x0


In [56]:
thresh = 0.01
optimizer = ps.STLSQ(threshold=thresh, alpha=1e-5, max_iter=50)
model = ps.SINDy(feature_library=generalized_library, optimizer=optimizer)
model.fit(data_2,  x_dot=data_dot,ensemble=True)


library_ensemble_coefs = model.coef_list
mean_library_ensemble = np.mean(library_ensemble_coefs, axis=0)
std_library_ensemble = np.std(library_ensemble_coefs, axis=0)

/home/ldoyle/packages/sklearn/linear_model/_ridge.py:212: LinAlgWarning: Ill-conditioned matrix (rcond=1.79429e-18): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
/home/ldoyle/packages/sklearn/linear_model/_ridge.py:212: LinAlgWarning: Ill-conditioned matrix (rcond=1.79429e-18): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
/home/ldoyle/packages/sklearn/linear_model/_ridge.py:212: LinAlgWarning: Ill-conditioned matrix (rcond=1.83169e-18): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
/home/ldoyle/packages/sklearn/linear_model/_ridge.py:212: LinAlgWarning: Ill-conditioned matrix (rcond=1.83169e-18): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
/home/ldoyle/packages/sklearn/linear_model/_ridge.py:212: LinAlgWarning: Ill-conditioned matrix (rcond=1.82496e-18): result may not be accurate.
  return linalg.solve(A

In [57]:
model.print()

(x0)' = -3.538 x0 + 13.582 sin(1 x0) + 0.249 cos(1 x0) + -0.248 1 + -3.538 x0 + 0.159 x0 sin(1 x0) + -6.505 x0 cos(1 x0) + -0.999 x0^2 sin(1 x0) + -0.037 x0^2 cos(1 x0)
(x1)' = 1.000 x1


In [40]:
data_2[:4,:4,0]

array([[-1.20231637, -1.18104781, -1.1589953 , -1.1361282 ],
       [-1.19182347, -1.17034013, -1.1480649 , -1.12496701],
       [-1.1810587 , -1.1593875 , -1.13691459, -1.11360881],
       [-1.17020462, -1.14826571, -1.12551329, -1.10191614]])